# 实验一：四份数据集分别做五折交叉检验
# 实验二：TCGA做训练集，ICGC做验证集

+ 1. 机器学习模型包括：

缩写|英文全称|中文全称
--|:--:|--:
NB|naive_bayes|朴素贝叶斯
KNN|k_nearest_neighbors|k近邻算法
LR|logistic_regression|逻辑回归
RF|random_forest|随机森林
DT|decision_tree|决策树
SVM|support_vector_machine|支持向量机

+ 2. 数据包括四个文件:

数据集|来源|解释|实验一|实验二
-|:-:|:-:|:-:|-:
lihcchol_normal.csv|TCGA|肝胆肿瘤vs癌旁|五折交叉检验|训练集1
LIRI-JP_lihc_normal.csv|ICGC|肝癌vs癌旁|五折交叉检验|验证集1
lihcchol_other4gi.csv|TCGA|肝胆肿瘤vs其它消化道肿瘤|五折交叉检验|训练集2
lihc_other4gi.csv|ICGC|肝癌vs其它消化道肿瘤|五折交叉检验|验证集2

+ 3. Feature选择:

features|APOA1|APCS|ARG1|LBP|FGA
-|-|-|-|-|-
1| √ | √ | √ | √ |
2| √ | √ | √ | √ | √ 

## 实验一：

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import json
import time
import random
import pickle5 as pickle
# from sklearn import metrics

from sklearn.model_selection import train_test_split 
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn import metrics

random.seed(0)

### 步骤一，导入数据：

In [15]:
# 读入数据
genes_lihc_other4gi = pd.read_csv("./data/7genes_lihc_other4gi.csv")
genes_lihcchol_normal = pd.read_csv("./data/7genes_lihcchol_normal.csv")
genes_lihcchol_other4gi = pd.read_csv("./data/7genes_lihcchol_other4gi.csv")
genes_LIRI_JP_lihc_normal = pd.read_csv("./data/7genes_LIRI-JP_lihc_normal.csv")

In [16]:
genes_lihc_other4gi.head(5) # 701 rows x 10 columns

,Cancer,labels,Hybridization.REF,APCS,APOA1,ARG1,FGA,C9,MBL2,LBP
0,PACA-CA,0,PCSI_0103_Pa_P_526,0.03061,0.060728,0.580261,0.033032,0.000000,0.0,0.000000
1,PACA-CA,0,MPCC_0024_Pa_C,0.00000,0.024033,0.000000,0.000000,0.007073,0.0,0.000000
2,PACA-CA,0,PCSI_0111_Pa_P_526,0.00000,0.289008,0.156345,0.013726,0.064629,0.0,0.000000
3,PACA-CA,0,PCSI_0101_Pa_P_526,12.83460,0.882541,0.000000,14.758200,0.334614,0.0,0.441031
4,PACA-CA,0,PCSI_0105_Pa_P_526,10.33700,0.000000,0.132646,1.440490,0.083736,0.0,0.008343


In [17]:
genes_lihcchol_normal.head(5) # 468 rows x 9 columns

,Hybridization.REF,labels,APCS,APOA1,ARG1,FGA,C9,MBL2,LBP
0,TCGA.BC.A10Q.11A.11R.A131.07,0,51087.5984,612479.3307,26083.6614,6.633907e+05,51895.6693,10295.2756,22239.1732
1,TCGA.BC.A10R.11A.11R.A131.07,0,44368.2076,586369.7799,22631.9854,7.049739e+05,43466.9278,6362.7462,42357.3950
2,TCGA.BC.A10T.11A.11R.A131.07,0,56170.8145,507402.7149,19135.7466,1.023110e+06,47368.7783,8099.5475,56819.0045
3,TCGA.BC.A10U.11A.11R.A131.07,0,52465.9484,525597.8857,26656.6375,7.114161e+05,36627.7699,4034.1533,39996.7473
4,TCGA.BC.A10W.11A.11R.A131.07,0,65673.5395,767685.5670,27333.3333,6.824210e+05,50238.8316,9118.5567,31841.9244


In [18]:
genes_lihcchol_other4gi.head(5) # 1516 rows x 9 columns

,Hybridization.REF,labels,APCS,APOA1,ARG1,FGA,C9,MBL2,LBP
0,TCGA.2V.A95S.01A.11R.A37K.07,1,1177.3905,285011.7674,395.5042,202874.0837,98.3874,354.4551,974.7516
1,TCGA.2Y.A9GS.01A.12R.A38B.07,1,10276.4909,84196.6486,26051.7496,86573.6816,3544.6033,1009.3642,10784.1301
2,TCGA.2Y.A9GT.01A.11R.A38B.07,1,45668.6842,197555.7050,17322.5734,502987.6709,43427.0343,10488.2313,25509.0787
3,TCGA.2Y.A9GU.01A.11R.A38B.07,1,45330.4861,309749.4244,8787.6614,379639.4848,4.1707,223.5494,3480.0307
4,TCGA.2Y.A9GV.01A.11R.A38B.07,1,28766.4975,266806.2606,17051.6074,466648.9002,35840.1015,6896.7851,32119.2893


In [19]:
genes_LIRI_JP_lihc_normal.head(5) # 445 rows x 10 columns

,Cancer,labels,Hybridization.REF,APCS,APOA1,ARG1,FGA,C9,MBL2,LBP
0,LIRI-JP,1,RK326_Cancer,793.569196,17779.961150,27.033219,390.064058,2.699527,1.144207,226.092886
1,LIRI-JP,1,RK337_Cancer,1672.534800,3026.380206,603.641114,2322.014128,12.548260,30.703011,101.736631
2,LIRI-JP,1,RK338_Cancer,522.103578,2429.628840,293.360524,2815.226203,0.117045,55.365247,60.151431
3,LIRI-JP,1,RK338_Cancer2,522.103578,2429.628840,293.360524,2815.226203,0.117045,55.365247,60.151431
4,LIRI-JP,1,RK305_Cancer,204.296083,8288.794400,95.237739,4681.866241,27.236537,25.407819,353.443858


### 步骤二，五折交叉检验，训练&评估：

### 2.1 构建函数

In [20]:
def cross_validation(data, features, model_save_path, evaluation_save_path, k=5, preprocess=True):
    """
    @data: 数据来源。可选lihcchol_normal，LIRI_JP_lihc_normal，lihcchol_other4gi，lihc_other4gi。
    @features: 参与分类的特征基因。
    @k: 交叉检验的值，默认为5。
    @preprocess: 是否对数据进行预处理，如归一化。默认为True。
    @model_save_path: 训练的模型保存保存路径。
    @evaluation_save_path: 训练的模型的评估结果保存路径。
    """
    # 获取X(feature), y(label).
    X = data[features].values
    y = data["labels"].values

    # 将数据进行预处理，此处预处理方式为归一化，
    if preprocess:
        min_max_scaler = preprocessing.MinMaxScaler()
        X = min_max_scaler.fit_transform(X)
        
    # 设置数据切割的次数，也即交叉检验的折数
    random.seed(0)
    kf = KFold(n_splits=k, shuffle = True, random_state=0)
    print(kf)
    
    # 存储评估结果
    methods = ["NB", "KNN", "LR", "RF", "DT","SVM"]
    # evaluation_metrics = ["precision", "recall", "f1", "auc", "accuracy_score", "roc_curve", "roc_auc_score"]
    evaluation_metrics = ["precision", "recall", "f1", "accuracy_score", "roc_auc_score", "auc"]
    evaluation_results = {}
    for m in methods:
        evaluation_results[m] = {}
        for e in evaluation_metrics:
            evaluation_results[m][e] = []
    
    # 开始训练
    n = 0
    for train_index, test_index in kf.split(X):
        n += 1
        # 准备数据X_train, X_test, y_train, y_test
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        
        # ================================================================================
        # 训练NB模型
        from sklearn.naive_bayes import GaussianNB
        NB_model = GaussianNB()
        NB_model.fit(X_train, y_train)
        # 评估NB模型
        NB_y_predict = NB_model.predict(X_test)
        evaluation_results["NB"]["precision"].append(metrics.precision_score(y_test, NB_y_predict))
        evaluation_results["NB"]["recall"].append(metrics.recall_score(y_test, NB_y_predict))
        evaluation_results["NB"]["f1"].append(metrics.f1_score(y_test, NB_y_predict))
        # evaluation_results["NB"]["auc"].append(metrics.auc(y_test, NB_y_predict))
        evaluation_results["NB"]["accuracy_score"].append(metrics.accuracy_score(y_test, NB_y_predict))
        # evaluation_results["NB"]["roc_curve"].append(metrics.roc_curve(y_test, NB_y_predict))
        evaluation_results["NB"]["roc_auc_score"].append(metrics.roc_auc_score(y_test, NB_y_predict))
        
        NB_y_predict_proba = NB_model.predict_proba(X_test)[:,1]
        fpr, tpr, thresholds = metrics.roc_curve(y_test, NB_y_predict_proba, pos_label=1)
        evaluation_results["NB"]["auc"].append(metrics.auc(fpr, tpr))
        
        # 保存NB模型
        pickle.dump(NB_model, open(os.path.join(model_save_path, "NB_{}.pkl".format(n)), 'wb'))
        # ================================================================================
        
        
        # ================================================================================
        # 训练KNN模型
        from sklearn.neighbors import KNeighborsClassifier  
        KNN_model = KNeighborsClassifier()  
        KNN_model.fit(X_train, y_train)
        # 评估KNN模型
        KNN_y_predict = KNN_model.predict(X_test)
        evaluation_results["KNN"]["precision"].append(metrics.precision_score(y_test, KNN_y_predict))
        evaluation_results["KNN"]["recall"].append(metrics.recall_score(y_test, KNN_y_predict))
        evaluation_results["KNN"]["f1"].append(metrics.f1_score(y_test, KNN_y_predict))
        # evaluation_results["KNN"]["auc"].append(metrics.auc(y_test, KNN_y_predict))
        evaluation_results["KNN"]["accuracy_score"].append(metrics.accuracy_score(y_test, KNN_y_predict))
        # evaluation_results["KNN"]["roc_curve"].append(metrics.roc_curve(y_test, KNN_y_predict))
        evaluation_results["KNN"]["roc_auc_score"].append(metrics.roc_auc_score(y_test, KNN_y_predict))
        
        KNN_y_predict_proba = KNN_model.predict_proba(X_test)[:,1]
        fpr, tpr, thresholds = metrics.roc_curve(y_test, KNN_y_predict_proba, pos_label=1)
        evaluation_results["KNN"]["auc"].append(metrics.auc(fpr, tpr))
        
        # 保存KNN模型
        pickle.dump(KNN_model, open(os.path.join(model_save_path, "KNN_{}.pkl".format(n)), 'wb'))
        # ================================================================================
        
        
        # ================================================================================
        # 训练LR模型
        from sklearn.linear_model import LogisticRegression  
        LR_model = LogisticRegression(penalty='l2', solver = "lbfgs")
        LR_model.fit(X_train, y_train)
        # 评估LR模型
        LR_y_predict = LR_model.predict(X_test)
        evaluation_results["LR"]["precision"].append(metrics.precision_score(y_test, LR_y_predict))
        evaluation_results["LR"]["recall"].append(metrics.recall_score(y_test, LR_y_predict))
        evaluation_results["LR"]["f1"].append(metrics.f1_score(y_test, LR_y_predict))
        # evaluation_results["LR"]["auc"].append(metrics.auc(y_test, LR_y_predict))
        evaluation_results["LR"]["accuracy_score"].append(metrics.accuracy_score(y_test, LR_y_predict))
        # evaluation_results["LR"]["roc_curve"].append(metrics.roc_curve(y_test, LR_y_predict))
        evaluation_results["LR"]["roc_auc_score"].append(metrics.roc_auc_score(y_test, LR_y_predict))
        
        LR_y_predict_proba = LR_model.predict_proba(X_test)[:,1]
        fpr, tpr, thresholds = metrics.roc_curve(y_test, LR_y_predict_proba, pos_label=1)
        evaluation_results["LR"]["auc"].append(metrics.auc(fpr, tpr))
        
        # 保存LR模型
        pickle.dump(LR_model, open(os.path.join(model_save_path, "LR_{}.pkl".format(n)), 'wb'))
        # ================================================================================
        
        
        # ================================================================================
        # 训练RF模型
        from sklearn.ensemble import RandomForestClassifier  
        RF_model = RandomForestClassifier(random_state=0)
        RF_model.fit(X_train, y_train)
        # 评估RF模型
        RF_y_predict = RF_model.predict(X_test)
        evaluation_results["RF"]["precision"].append(metrics.precision_score(y_test, RF_y_predict))
        evaluation_results["RF"]["recall"].append(metrics.recall_score(y_test, RF_y_predict))
        evaluation_results["RF"]["f1"].append(metrics.f1_score(y_test, RF_y_predict))
        # evaluation_results["RF"]["auc"].append(metrics.auc(y_test, RF_y_predict))
        evaluation_results["RF"]["accuracy_score"].append(metrics.accuracy_score(y_test, RF_y_predict))
        # evaluation_results["RF"]["roc_curve"].append(metrics.roc_curve(y_test, RF_y_predict))
        evaluation_results["RF"]["roc_auc_score"].append(metrics.roc_auc_score(y_test, RF_y_predict))
        
        RF_y_predict_proba = RF_model.predict_proba(X_test)[:,1]
        fpr, tpr, thresholds = metrics.roc_curve(y_test, RF_y_predict_proba, pos_label=1)
        evaluation_results["RF"]["auc"].append(metrics.auc(fpr, tpr))
        
        # 保存RF模型
        pickle.dump(RF_model, open(os.path.join(model_save_path, "RF_{}.pkl".format(n)), 'wb'))
        # ================================================================================
        
        
        # ================================================================================
        # 训练DT模型
        from sklearn import tree  
        DT_model = tree.DecisionTreeClassifier(random_state=0)
        DT_model.fit(X_train, y_train)
        # 评估DT模型
        DT_y_predict = DT_model.predict(X_test)
        evaluation_results["DT"]["precision"].append(metrics.precision_score(y_test, DT_y_predict))
        evaluation_results["DT"]["recall"].append(metrics.recall_score(y_test, DT_y_predict))
        evaluation_results["DT"]["f1"].append(metrics.f1_score(y_test, DT_y_predict))
        # evaluation_results["DT"]["auc"].append(metrics.auc(y_test, DT_y_predict))
        evaluation_results["DT"]["accuracy_score"].append(metrics.accuracy_score(y_test, DT_y_predict))
        # evaluation_results["DT"]["roc_curve"].append(metrics.roc_curve(y_test, DT_y_predict))
        evaluation_results["DT"]["roc_auc_score"].append(metrics.roc_auc_score(y_test, DT_y_predict))
        
        DT_y_predict_proba = DT_model.predict_proba(X_test)[:,1]
        fpr, tpr, thresholds = metrics.roc_curve(y_test, DT_y_predict_proba, pos_label=1)
        evaluation_results["DT"]["auc"].append(metrics.auc(fpr, tpr))
        
        # 保存DT模型
        pickle.dump(DT_model, open(os.path.join(model_save_path, "DT_{}.pkl".format(n)), 'wb'))
        # ================================================================================
        
        
        # ================================================================================
        # 训练SVM模型
        from sklearn.svm import SVC  
        SVM_model = SVC(kernel='rbf', gamma='auto')
        SVM_model.fit(X_train, y_train)
        # 评估SVM模型
        SVM_y_predict = SVM_model.predict(X_test)
        evaluation_results["SVM"]["precision"].append(metrics.precision_score(y_test, SVM_y_predict))
        evaluation_results["SVM"]["recall"].append(metrics.recall_score(y_test, SVM_y_predict))
        evaluation_results["SVM"]["f1"].append(metrics.f1_score(y_test, SVM_y_predict))
        # evaluation_results["SVM"]["auc"].append(metrics.auc(y_test, SVM_y_predict))
        evaluation_results["SVM"]["accuracy_score"].append(metrics.accuracy_score(y_test, SVM_y_predict))
        # evaluation_results["SVM"]["roc_curve"].append(metrics.roc_curve(y_test, SVM_y_predict))
        evaluation_results["SVM"]["roc_auc_score"].append(metrics.roc_auc_score(y_test, SVM_y_predict))
        
        fpr, tpr, thresholds = metrics.roc_curve(y_test, SVM_model.decision_function(X_test), pos_label=1)
        evaluation_results["SVM"]["auc"].append(metrics.auc(fpr, tpr))
        
        # 保存SVM模型
        pickle.dump(SVM_model, open(os.path.join(model_save_path, "SVM_{}.pkl".format(n)), 'wb'))
        # ================================================================================
        
        
    # 将五折的各个评价指标进行平均。
    evaluation_results_mean={}
    for m in methods:
        evaluation_results_mean[m] = {}
        for e in evaluation_metrics:
            evaluation_results_mean[m][e] = np.mean(evaluation_results[m][e])
    
    # 保存评估结果
    with open(os.path.join(evaluation_save_path, "a_results.json"), 'w') as outf:
        json.dump(evaluation_results_mean,outf,ensure_ascii=False, indent=4)
        outf.write('\n')
    
    return evaluation_results_mean

### 2.2 不同数据，不同feature，进行训练

In [22]:
# gene选择"APOA1", "APCS", "ARG1", "LBP"， 数据为lihc_other4gi
data = genes_lihc_other4gi
features = ["APOA1", "APCS", "ARG1", "LBP"]
model_save_path = "./model/cross_validation/4gene_genes_lihc_other4gi/"
evaluation_save_path = "./model/cross_validation/4gene_genes_lihc_other4gi/"
evaluation_results_mean = cross_validation(data, features, model_save_path, evaluation_save_path)
pd.DataFrame.from_dict(evaluation_results_mean, orient='index')

KFold(n_splits=5, random_state=0, shuffle=True)


,precision,recall,f1,accuracy_score,roc_auc_score,auc
DT,0.974233,0.959544,0.966700,0.962898,0.963492,0.963492
KNN,0.962240,0.960033,0.960734,0.955765,0.955529,0.989579
LR,0.994118,0.846122,0.913756,0.907295,0.919804,0.990581
NB,0.984125,0.950890,0.967123,0.962928,0.965717,0.987776
RF,0.985016,0.984598,0.984763,0.982867,0.982192,0.993429
SVM,0.996774,0.776507,0.872662,0.868794,0.886558,0.990705


In [23]:
# gene选择"APOA1", "APCS", "ARG1", "LBP"， 数据为lihcchol_normal
data = genes_lihcchol_normal
features = ["APOA1", "APCS", "ARG1", "LBP"]
model_save_path = "./model/cross_validation/4gene_genes_lihcchol_normal/"
evaluation_save_path = "./model/cross_validation/4gene_genes_lihcchol_normal/"
evaluation_results_mean = cross_validation(data, features, model_save_path, evaluation_save_path)
pd.DataFrame.from_dict(evaluation_results_mean, orient='index')

KFold(n_splits=5, random_state=0, shuffle=True)


,precision,recall,f1,accuracy_score,roc_auc_score,auc
DT,0.947562,0.956306,0.951554,0.914573,0.797237,0.797237
KNN,0.975211,0.953720,0.964136,0.938069,0.898716,0.961391
LR,0.879698,0.980650,0.927112,0.865454,0.529128,0.923917
NB,0.966082,0.973444,0.969501,0.946648,0.872082,0.962687
RF,0.964249,0.973511,0.968462,0.944452,0.863862,0.967267
SVM,0.873942,1.000000,0.932602,0.873942,0.500000,0.949985


In [24]:
# gene选择"APOA1", "APCS", "ARG1", "LBP"， 数据为lihcchol_other4gi
data = genes_lihcchol_other4gi
features = ["APOA1", "APCS", "ARG1", "LBP"]
model_save_path = "./model/cross_validation/4gene_genes_lihcchol_other4gi/"
evaluation_save_path = "./model/cross_validation/4gene_genes_lihcchol_other4gi/"
evaluation_results_mean = cross_validation(data, features, model_save_path, evaluation_save_path)
pd.DataFrame.from_dict(evaluation_results_mean, orient='index')

KFold(n_splits=5, random_state=0, shuffle=True)


,precision,recall,f1,accuracy_score,roc_auc_score,auc
DT,0.962863,0.943370,0.952817,0.974937,0.964878,0.964878
KNN,0.986799,0.943471,0.964614,0.981529,0.969499,0.981184
LR,0.995652,0.664430,0.796533,0.908970,0.831780,0.994227
NB,0.974757,0.944270,0.959143,0.978235,0.967627,0.988227
RF,0.979132,0.953740,0.966180,0.982189,0.973301,0.993748
SVM,0.996154,0.746709,0.853339,0.931397,0.872919,0.994175


In [25]:
# gene选择"APOA1", "APCS", "ARG1", "LBP"， 数据为LIRI_JP_lihc_normal
data = genes_LIRI_JP_lihc_normal
features = ["APOA1", "APCS", "ARG1", "LBP"]
model_save_path = "./model/cross_validation/4gene_genes_LIRI_JP_lihc_normal/"
evaluation_save_path = "./model/cross_validation/4gene_genes_LIRI_JP_lihc_normal/"
evaluation_results_mean = cross_validation(data, features, model_save_path, evaluation_save_path)
pd.DataFrame.from_dict(evaluation_results_mean, orient='index')

KFold(n_splits=5, random_state=0, shuffle=True)


,precision,recall,f1,accuracy_score,roc_auc_score,auc
DT,0.828957,0.860071,0.843161,0.826966,0.823165,0.823165
KNN,0.907974,0.801829,0.850704,0.847191,0.850842,0.905418
LR,0.782088,0.831282,0.803823,0.779775,0.774260,0.834565
NB,0.900266,0.816198,0.853360,0.849438,0.849391,0.895365
RF,0.882622,0.876872,0.878103,0.867416,0.866458,0.923400
SVM,0.836266,0.781969,0.805626,0.795506,0.797439,0.848665


In [26]:
# gene选择"APOA1", "APCS", "ARG1", "LBP", "FGA"， 数据为lihc_other4gi
data = genes_lihc_other4gi
features = ["APOA1", "APCS", "ARG1", "LBP", "FGA"]
model_save_path = "./model/cross_validation/5gene_genes_lihc_other4gi/"
evaluation_save_path = "./model/cross_validation/5gene_genes_lihc_other4gi/"
evaluation_results_mean = cross_validation(data, features, model_save_path, evaluation_save_path)
pd.DataFrame.from_dict(evaluation_results_mean, orient='index')

KFold(n_splits=5, random_state=0, shuffle=True)


,precision,recall,f1,accuracy_score,roc_auc_score,auc
DT,0.976935,0.962283,0.969421,0.965755,0.966355,0.966355
KNN,0.968872,0.962348,0.965468,0.961459,0.962218,0.986808
LR,0.988590,0.865789,0.922824,0.915876,0.926222,0.990034
NB,0.976297,0.950785,0.963257,0.958632,0.960984,0.984120
RF,0.987238,0.979389,0.983228,0.981439,0.981406,0.994289
SVM,0.993651,0.801656,0.886960,0.881641,0.897571,0.990159


In [27]:
# gene选择"APOA1", "APCS", "ARG1", "LBP", "FGA"， 数据为lihcchol_normal
data = genes_lihcchol_normal
features = ["APOA1", "APCS", "ARG1", "LBP", "FGA"]
model_save_path = "./model/cross_validation/5gene_genes_lihcchol_normal/"
evaluation_save_path = "./model/cross_validation/5gene_genes_lihcchol_normal/"
evaluation_results_mean = cross_validation(data, features, model_save_path, evaluation_save_path)
pd.DataFrame.from_dict(evaluation_results_mean, orient='index')

KFold(n_splits=5, random_state=0, shuffle=True)


,precision,recall,f1,accuracy_score,roc_auc_score,auc
DT,0.966022,0.965895,0.965781,0.940243,0.867746,0.867746
KNN,0.992172,0.948514,0.969809,0.948707,0.951729,0.986545
LR,0.889578,0.978209,0.931383,0.874011,0.573293,0.946761
NB,0.975086,0.958625,0.966693,0.942347,0.896651,0.970208
RF,0.971396,0.983096,0.976949,0.959414,0.895150,0.984330
SVM,0.873942,1.000000,0.932602,0.873942,0.500000,0.960270


In [28]:
# gene选择"APOA1", "APCS", "ARG1", "LBP", "FGA"， 数据为lihcchol_other4gi
data = genes_lihcchol_other4gi
features = ["APOA1", "APCS", "ARG1", "LBP", "FGA"]
model_save_path = "./model/cross_validation/5gene_genes_lihcchol_other4gi/"
evaluation_save_path = "./model/cross_validation/5gene_genes_lihcchol_other4gi/"
evaluation_results_mean = cross_validation(data, features, model_save_path, evaluation_save_path)
pd.DataFrame.from_dict(evaluation_results_mean, orient='index')

KFold(n_splits=5, random_state=0, shuffle=True)


,precision,recall,f1,accuracy_score,roc_auc_score,auc
DT,0.957206,0.958630,0.957844,0.976911,0.971078,0.971078
KNN,0.990157,0.945797,0.967447,0.982849,0.971059,0.979623
LR,0.996078,0.698904,0.821234,0.918206,0.849017,0.994404
NB,0.977311,0.941567,0.959011,0.978235,0.966711,0.988195
RF,0.982642,0.958041,0.970149,0.984169,0.975831,0.993447
SVM,0.996364,0.765879,0.865531,0.936671,0.882504,0.994372


In [29]:
# gene选择"APOA1", "APCS", "ARG1", "LBP", "FGA"， 数据为LIRI_JP_lihc_normal
data = genes_LIRI_JP_lihc_normal
features = ["APOA1", "APCS", "ARG1", "LBP", "FGA"]
model_save_path = "./model/cross_validation/5gene_genes_LIRI_JP_lihc_normal/"
evaluation_save_path = "./model/cross_validation/5gene_genes_LIRI_JP_lihc_normal/"
evaluation_results_mean = cross_validation(data, features, model_save_path, evaluation_save_path)
pd.DataFrame.from_dict(evaluation_results_mean, orient='index')

KFold(n_splits=5, random_state=0, shuffle=True)


,precision,recall,f1,accuracy_score,roc_auc_score,auc
DT,0.815292,0.822804,0.816737,0.800000,0.798869,0.798869
KNN,0.912727,0.809862,0.857738,0.853933,0.858122,0.907397
LR,0.866521,0.850935,0.857648,0.847191,0.846393,0.879509
NB,0.930243,0.811423,0.864895,0.865169,0.868174,0.893226
RF,0.889587,0.880100,0.883474,0.874157,0.873447,0.926414
SVM,0.898796,0.798084,0.845146,0.840449,0.844241,0.884620


## 实验二：

In [42]:
def train_validation(train_data, validation_data, features, model_save_path, evaluation_save_path, k=5, preprocess=True):
    """
    @train_data, 训练数据，包括lihcchol_normal，lihcchol_other4gi
    @validation_data: 验证数据，包括LIRI_JP_lihc_normal，lihc_other4gi。
    @features: 参与分类的特征基因。
    @k: 交叉检验的值，默认为5。
    @preprocess: 是否对数据进行预处理，如归一化。默认为True。
    @model_save_path: 训练的模型保存保存路径。
    @evaluation_save_path: 训练的模型的评估结果保存路径。
    """
    # 获取X(feature), y(label).
    X = data[features].values
    y = data["labels"].values

    # 将数据进行预处理，此处预处理方式为归一化，
    if preprocess:
        min_max_scaler = preprocessing.MinMaxScaler()
        X = min_max_scaler.fit_transform(X)
    
    # 存储评估结果
    methods = ["NB", "KNN", "LR", "RF", "DT","SVM"]
    # evaluation_metrics = ["precision", "recall", "f1", "auc", "accuracy_score", "roc_curve", "roc_auc_score"]
    evaluation_metrics = ["precision", "recall", "f1", "accuracy_score", "roc_auc_score", "auc"]
    evaluation_results = {}
    for m in methods:
        evaluation_results[m] = {}
        for e in evaluation_metrics:
            evaluation_results[m][e] = 0.0
    
    # 准备数据X_train, X_test, y_train, y_test
    X_train, X_test = train_data[features].values, validation_data[features].values
    y_train, y_test = train_data["labels"].values, validation_data["labels"].values

        
    # ================================================================================
    # 训练NB模型
    from sklearn.naive_bayes import GaussianNB
    NB_model = GaussianNB()
    NB_model.fit(X_train, y_train)
    # 评估NB模型
    NB_y_predict = NB_model.predict(X_test)
    evaluation_results["NB"]["precision"] = metrics.precision_score(y_test, NB_y_predict)
    evaluation_results["NB"]["recall"] = metrics.recall_score(y_test, NB_y_predict)
    evaluation_results["NB"]["f1"] = metrics.f1_score(y_test, NB_y_predict)
    # evaluation_results["NB"]["auc"] = metrics.auc(y_test, NB_y_predict))
    evaluation_results["NB"]["accuracy_score"] = metrics.accuracy_score(y_test, NB_y_predict)
    # evaluation_results["NB"]["roc_curve"] = metrics.roc_curve(y_test, NB_y_predict))
    evaluation_results["NB"]["roc_auc_score"] = metrics.roc_auc_score(y_test, NB_y_predict)

    NB_y_predict_proba = NB_model.predict_proba(X_test)[:,1]
    fpr, tpr, thresholds = metrics.roc_curve(y_test, NB_y_predict_proba, pos_label=1)
    evaluation_results["NB"]["auc"] = metrics.auc(fpr, tpr)

    # 保存NB模型
    pickle.dump(NB_model, open(os.path.join(model_save_path, "NB.pkl"), 'wb'))
    # ================================================================================
    
    
    # ================================================================================
    # 训练KNN模型
    from sklearn.neighbors import KNeighborsClassifier  
    KNN_model = KNeighborsClassifier()  
    KNN_model.fit(X_train, y_train)
    # 评估KNN模型
    KNN_y_predict = KNN_model.predict(X_test)
    evaluation_results["KNN"]["precision"] = metrics.precision_score(y_test, KNN_y_predict)
    evaluation_results["KNN"]["recall"] = metrics.recall_score(y_test, KNN_y_predict)
    evaluation_results["KNN"]["f1"] = metrics.f1_score(y_test, KNN_y_predict)
    # evaluation_results["KNN"]["auc"] = metrics.auc(y_test, KNN_y_predict))
    evaluation_results["KNN"]["accuracy_score"] = metrics.accuracy_score(y_test, KNN_y_predict)
    # evaluation_results["KNN"]["roc_curve"] = metrics.roc_curve(y_test, KNN_y_predict))
    evaluation_results["KNN"]["roc_auc_score"] = metrics.roc_auc_score(y_test, KNN_y_predict)

    KNN_y_predict_proba = KNN_model.predict_proba(X_test)[:,1]
    fpr, tpr, thresholds = metrics.roc_curve(y_test, KNN_y_predict_proba, pos_label=1)
    evaluation_results["KNN"]["auc"] = metrics.auc(fpr, tpr)

    # 保存KNN模型
    pickle.dump(KNN_model, open(os.path.join(model_save_path, "KNN.pkl"), 'wb'))
    # ================================================================================


    # ================================================================================
    # 训练LR模型
    from sklearn.linear_model import LogisticRegression  
    LR_model = LogisticRegression(penalty='l2', solver = "lbfgs")
    LR_model.fit(X_train, y_train)
    # 评估LR模型
    LR_y_predict = LR_model.predict(X_test)
    evaluation_results["LR"]["precision"] = metrics.precision_score(y_test, LR_y_predict)
    evaluation_results["LR"]["recall"] = metrics.recall_score(y_test, LR_y_predict)
    evaluation_results["LR"]["f1"] = metrics.f1_score(y_test, LR_y_predict)
    # evaluation_results["LR"]["auc"] = metrics.auc(y_test, LR_y_predict))
    evaluation_results["LR"]["accuracy_score"] = metrics.accuracy_score(y_test, LR_y_predict)
    # evaluation_results["LR"]["roc_curve"] = metrics.roc_curve(y_test, LR_y_predict))
    evaluation_results["LR"]["roc_auc_score"] = metrics.roc_auc_score(y_test, LR_y_predict)

    LR_y_predict_proba = LR_model.predict_proba(X_test)[:,1]
    fpr, tpr, thresholds = metrics.roc_curve(y_test, LR_y_predict_proba, pos_label=1)
    evaluation_results["LR"]["auc"] = metrics.auc(fpr, tpr)

    # 保存LR模型
    pickle.dump(LR_model, open(os.path.join(model_save_path, "LR.pkl"), 'wb'))
    # ================================================================================


    # ================================================================================
    # 训练RF模型
    from sklearn.ensemble import RandomForestClassifier  
    RF_model = RandomForestClassifier(random_state=0)
    RF_model.fit(X_train, y_train)
    # 评估RF模型
    RF_y_predict = RF_model.predict(X_test)
    evaluation_results["RF"]["precision"] = metrics.precision_score(y_test, RF_y_predict)
    evaluation_results["RF"]["recall"] = metrics.recall_score(y_test, RF_y_predict)
    evaluation_results["RF"]["f1"] = metrics.f1_score(y_test, RF_y_predict)
    # evaluation_results["RF"]["auc"] = metrics.auc(y_test, RF_y_predict))
    evaluation_results["RF"]["accuracy_score"] = metrics.accuracy_score(y_test, RF_y_predict)
    # evaluation_results["RF"]["roc_curve"] = metrics.roc_curve(y_test, RF_y_predict))
    evaluation_results["RF"]["roc_auc_score"] = metrics.roc_auc_score(y_test, RF_y_predict)

    RF_y_predict_proba = RF_model.predict_proba(X_test)[:,1]
    fpr, tpr, thresholds = metrics.roc_curve(y_test, RF_y_predict_proba, pos_label=1)
    evaluation_results["RF"]["auc"] = metrics.auc(fpr, tpr)

    # 保存RF模型
    pickle.dump(RF_model, open(os.path.join(model_save_path, "RF.pkl"), 'wb'))
    # ================================================================================


    # ================================================================================
    # 训练DT模型
    from sklearn import tree  
    DT_model = tree.DecisionTreeClassifier(random_state=0)
    DT_model.fit(X_train, y_train)
    # 评估DT模型
    DT_y_predict = DT_model.predict(X_test)
    evaluation_results["DT"]["precision"] = metrics.precision_score(y_test, DT_y_predict)
    evaluation_results["DT"]["recall"] = metrics.recall_score(y_test, DT_y_predict)
    evaluation_results["DT"]["f1"] = metrics.f1_score(y_test, DT_y_predict)
    # evaluation_results["DT"]["auc"] = metrics.auc(y_test, DT_y_predict))
    evaluation_results["DT"]["accuracy_score"] = metrics.accuracy_score(y_test, DT_y_predict)
    # evaluation_results["DT"]["roc_curve"] = metrics.roc_curve(y_test, DT_y_predict))
    evaluation_results["DT"]["roc_auc_score"] = metrics.roc_auc_score(y_test, DT_y_predict)

    DT_y_predict_proba = DT_model.predict_proba(X_test)[:,1]
    fpr, tpr, thresholds = metrics.roc_curve(y_test, DT_y_predict_proba, pos_label=1)
    evaluation_results["DT"]["auc"] = metrics.auc(fpr, tpr)

    # 保存DT模型
    pickle.dump(DT_model, open(os.path.join(model_save_path, "DT.pkl"), 'wb'))
    # ================================================================================


    # ================================================================================
    # 训练SVM模型
    from sklearn.svm import SVC  
    SVM_model = SVC(kernel='rbf', gamma='auto')
    SVM_model.fit(X_train, y_train)
    # 评估SVM模型
    SVM_y_predict = SVM_model.predict(X_test)
    evaluation_results["SVM"]["precision"] = metrics.precision_score(y_test, SVM_y_predict)
    evaluation_results["SVM"]["recall"] = metrics.recall_score(y_test, SVM_y_predict)
    evaluation_results["SVM"]["f1"] = metrics.f1_score(y_test, SVM_y_predict)
    # evaluation_results["SVM"]["auc"] = metrics.auc(y_test, SVM_y_predict))
    evaluation_results["SVM"]["accuracy_score"] = metrics.accuracy_score(y_test, SVM_y_predict)
    # evaluation_results["SVM"]["roc_curve"] = metrics.roc_curve(y_test, SVM_y_predict))
    evaluation_results["SVM"]["roc_auc_score"] = metrics.roc_auc_score(y_test, SVM_y_predict)

    fpr, tpr, thresholds = metrics.roc_curve(y_test, SVM_model.decision_function(X_test), pos_label=1)
    evaluation_results["SVM"]["auc"] = metrics.auc(fpr, tpr)

    # 保存SVM模型
    pickle.dump(SVM_model, open(os.path.join(model_save_path, "SVM.pkl"), 'wb'))
    # ================================================================================

    
    # 保存评估结果
    with open(os.path.join(evaluation_save_path, "a_results.json"), 'w') as outf:
        json.dump(evaluation_results,outf,ensure_ascii=False, indent=4)
        outf.write('\n')
    
    return evaluation_results

In [49]:
# gene选择"APOA1", "APCS", "ARG1", "LBP"， 训练数据为lihcchol_normal，验证数据为LIRI_JP_lihc_normal
train_data = genes_lihcchol_normal
validation_data = genes_LIRI_JP_lihc_normal
features = ["APOA1", "APCS", "ARG1", "LBP"]
model_save_path = "./model/train_validation/4gene_lihcchol_normal & LIRI_JP_lihc_normal/"
evaluation_save_path = "./model/train_validation/4gene_lihcchol_normal & LIRI_JP_lihc_normal/"
evaluation_results_mean = train_validation(train_data, validation_data, features, model_save_path, evaluation_save_path)
pd.DataFrame.from_dict(evaluation_results_mean, orient='index')

,precision,recall,f1,accuracy_score,roc_auc_score,auc
DT,0.546067,1.000000,0.706395,0.546067,0.50000,0.500000
KNN,0.546067,1.000000,0.706395,0.546067,0.50000,0.500000
LR,0.541096,0.975309,0.696035,0.534831,0.49013,0.418307
NB,0.546067,1.000000,0.706395,0.546067,0.50000,0.786008
RF,0.546067,1.000000,0.706395,0.546067,0.50000,0.500000
SVM,0.546067,1.000000,0.706395,0.546067,0.50000,0.506173


In [53]:
# gene选择"APOA1", "APCS", "ARG1", "LBP"，FGA， 训练数据为lihcchol_other4gi，验证数据为lihc_other4gi
train_data = genes_lihcchol_other4gi
validation_data = genes_lihc_other4gi
features = ["APOA1", "APCS", "ARG1", "LBP"]
model_save_path = "./model/train_validation/4gene_lihcchol_other4gi & lihc_other4gi/"
evaluation_save_path = "./model/train_validation/4gene_lihcchol_other4gi & lihc_other4gi/"
evaluation_results_mean = train_validation(train_data, validation_data, features, model_save_path, evaluation_save_path)
pd.DataFrame.from_dict(evaluation_results_mean, orient='index')

C:\Users\310272293\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\310272293\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1,accuracy_score,roc_auc_score,auc
DT,0.990991,0.816832,0.895522,0.890157,0.903365,0.903365
KNN,1.000000,0.014851,0.029268,0.432240,0.507426,0.694086
LR,0.000000,0.000000,0.000000,0.423680,0.500000,0.990007
NB,1.000000,0.002475,0.004938,0.425107,0.501238,0.924467
RF,0.993610,0.769802,0.867503,0.864479,0.881534,0.973681
SVM,0.000000,0.000000,0.000000,0.423680,0.500000,0.984378


In [54]:
# gene选择"APOA1", "APCS", "ARG1", "LBP"， 训练数据为lihcchol_normal，验证数据为LIRI_JP_lihc_normal
train_data = genes_lihcchol_normal
validation_data = genes_LIRI_JP_lihc_normal
features = ["APOA1", "APCS", "ARG1", "LBP", "FGA"]
model_save_path = "./model/train_validation/5gene_lihcchol_normal & LIRI_JP_lihc_normal/"
evaluation_save_path = "./model/train_validation/5gene_lihcchol_normal & LIRI_JP_lihc_normal/"
evaluation_results_mean = train_validation(train_data, validation_data, features, model_save_path, evaluation_save_path)
pd.DataFrame.from_dict(evaluation_results_mean, orient='index')

,precision,recall,f1,accuracy_score,roc_auc_score,auc
DT,0.546067,1.000000,0.706395,0.546067,0.500000,0.500000
KNN,0.546067,1.000000,0.706395,0.546067,0.500000,0.500000
LR,0.535632,0.958848,0.687316,0.523596,0.479424,0.436336
NB,0.546067,1.000000,0.706395,0.546067,0.500000,0.795288
RF,0.546067,1.000000,0.706395,0.546067,0.500000,0.500000
SVM,0.546067,1.000000,0.706395,0.546067,0.500000,0.504115


In [52]:
# gene选择"APOA1", "APCS", "ARG1", "LBP"，FGA, 训练数据为lihcchol_other4gi，验证数据为lihc_other4gi
train_data = genes_lihcchol_other4gi
validation_data = genes_lihc_other4gi
features = ["APOA1", "APCS", "ARG1", "LBP", "FGA"]
model_save_path = "./model/train_validation/5gene_lihcchol_other4gi & lihc_other4gi/"
evaluation_save_path = "./model/train_validation/5gene_lihcchol_other4gi & lihc_other4gi/"
evaluation_results_mean = train_validation(train_data, validation_data, features, model_save_path, evaluation_save_path)
pd.DataFrame.from_dict(evaluation_results_mean, orient='index')

C:\Users\310272293\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\310272293\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1,accuracy_score,roc_auc_score,auc
DT,0.986348,0.715347,0.829268,0.830243,0.850939,0.850939
KNN,1.000000,0.012376,0.024450,0.430813,0.506188,0.729048
LR,0.000000,0.000000,0.000000,0.423680,0.500000,0.990057
NB,1.000000,0.002475,0.004938,0.425107,0.501238,0.928076
RF,1.000000,0.613861,0.760736,0.777461,0.806931,0.976819
SVM,0.000000,0.000000,0.000000,0.423680,0.500000,0.953666
